In [ ]:
import os
import uuid

from IPython.display import display

import folium
from folium.plugins import MarkerCluster

import openrouteservice

import time
import pandas as pd
import geopandas as gpd
import fiona as fn
from shapely.geometry import shape, mapping
from shapely.ops import cascaded_union

from tqdm.notebook import tqdm

In [ ]:
#add stop geometries
stations = "/home/dabanto/Downloads/london_centroids_uid.gpkg"
df = gpd.read_file(stations)

In [ ]:
df  = df.to_crs({'init': 'epsg:4326'})

In [ ]:
from tqdm.notebook import tqdm
from openrouteservice import client 
import geopandas as gpd

normal_isochrones = []
err_iso = []


profile = 'driving-car'
isochrones_parameters = {
    'profile': profile,
    'range_type': 'distance',
    'range': [13640],
}




ors_client = client.Client(base_url='http://localhost:8081/ors')

# Use tqdm to wrap the iteration over the GeoDataFrame
for _, row in tqdm(df.iterrows(), total=len(df)):

    try:
        coordinate = (row.geometry.x, row.geometry.y)
        stop_id = row['uuid']  # Extract stop_id for the current row
    
        # Calculate isochrone for the single coordinate
        isochrones_parameters['locations'] = [coordinate]  # Ensure it's a list with one coordinate
        isochrone_normal = ors_client.isochrones(**isochrones_parameters)
        isochrone_normal_df = gpd.GeoDataFrame().from_features(isochrone_normal)
        # Add stop_id column to the isochrone GeoDataFrame
        isochrone_normal_df['uuid'] = stop_id
        # Append the single isochrone to the list
        normal_isochrones.append(isochrone_normal_df)
        
    except:
        print(row['uuid'])
        err_iso.append(row)
        continue

In [ ]:
isochrones_all = pd.concat(normal_isochrones).set_crs(epsg=4326)
isochrones_all

isochrones_all.to_file("isochrones_all.gpkg")